In [4]:
from skt.ye import get_spark
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
import sys
from datetime import datetime, timedelta

In [5]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client, \
                    bq_to_df

In [1]:
execution_dt = datetime.strptime(current_dt, '%Y%m%d')
execution_date_ago = (execution_dt - timedelta(days=1)).strftime('%Y/%m/%d')
execution_dt = execution_dt.strftime('%Y-%m-%d')
print(f'excution_dt: {execution_dt}')
print(f'execution_dt_ago: {execution_date_ago}')

# 음악 홈 쿼리

In [159]:
# query = f"""
# WITH RAW_HOME_LOG AS (
#     SELECT * 
#     FROM(
#         SELECT  JSON_EXTRACT(responseData, '$.transactionId') as transactionId,
#                 JSON_EXTRACT_ARRAY(JSON_EXTRACT(responseData, '$.sectionList')) as sectionList,
#                 JSON_EXTRACT(responseData, '$.size') as cardSize,
#                 JSON_EXTRACT(responseData, '$.elapsed') as elapsed,
#                 JSON_EXTRACT(responseData, '$.header') as header,
#                 request,
#                 response,
#                 responseStatusCode,
#                 responseCode,
#                 timestamp
#         FROM(
#             SELECT  JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.data') AS responseData,
#                     JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.statusCode') AS responseStatusCode,
#                     JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.code') AS responseCode,
#                     JSON_VALUE(log, '$.requestBody') AS request,
#                     JSON_VALUE(log, '$.responseBody') AS response,
#                     timestamp
#             FROM `skt-datahub.apollo_log.home_aggregator_prd` 
#             WHERE TIMESTAMP_TRUNC(timestamp, day) = TIMESTAMP('{execution_dt}')
#         )
#         WHERE JSON_EXTRACT(responseData, '$.header') is null --music home log 뽑는 조건
#     ),  UNNEST(sectionList) sectionList_item
# ),
# PREPROCESS_FIRST AS (
#     SELECT  transactionId,
#             Cast(cardSize as INT64) as cardSize,
#             responseStatusCode,
#             responseCode,
#             ARRAY_AGG(sectionInfo IGNORE NULLS) as cardList,
#             ARRAY_AGG(sectionTitle IGNORE NULLS) as cardTitleList,
#             IF (MIN(cardIndex)>0, True, FALSE) as cold_start,
#             MIN(cardIndex) as smallestIndex,
#             domain,
#             timestamp,
#             DATE(TIMESTAMP_TRUNC(timestamp, day)) as dt
            
#     FROM (
#         SELECT  transactionId,
#                 JSON_EXTRACT(sectionList_item, '$.title') as sectionTitle,
#                 JSON_EXTRACT(sectionList_item, '$.info') as sectionInfo,
#                 CAST(JSON_EXTRACT(JSON_EXTRACT(sectionList_item, '$.info'), '$.index') as INT64) as cardIndex,
#                 cardSize,
#                 JSON_EXTRACT(sectionList_item, '$.cardList') as sectioncardList,
#                 If(header is null, 'music_home', header) as domain,
#                 responseStatusCode,
#                 responseCode,
#                 timestamp
#         FROM RAW_HOME_LOG
#     ) 
#     GROUP BY transactionId, cardSize, responseStatusCode, responseCode, domain, timestamp
# )

# SELECT *
# FROM PREPROCESS_FIRST
# WHERE transactionId is not Null
# """

In [100]:
query = f"""
WITH RAW_HOME_LOG AS (
    SELECT * 
    FROM(
        SELECT  JSON_EXTRACT(responseData, '$.transactionId') as transactionId,
                JSON_EXTRACT_ARRAY(JSON_EXTRACT(responseData, '$.sectionList')) as sectionList,
                JSON_EXTRACT(responseData, '$.size') as cardSize,
                JSON_EXTRACT(responseData, '$.elapsed') as elapsed,
                JSON_EXTRACT(responseData, '$.header') as header,
                request,
                response,
                responseStatusCode,
                responseCode,
                timestamp
        FROM(
            SELECT  JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.data') AS responseData,
                    JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.statusCode') AS responseStatusCode,
                    JSON_EXTRACT(JSON_VALUE(log, '$.responseBody'),'$.code') AS responseCode,
                    JSON_VALUE(log, '$.requestBody') AS request,
                    JSON_VALUE(log, '$.responseBody') AS response,
                    timestamp
            FROM `skt-datahub.apollo_log.home_aggregator_prd` 
            WHERE TIMESTAMP_TRUNC(timestamp, day) = TIMESTAMP('{execution_dt}')
        )
        WHERE JSON_EXTRACT(responseData, '$.header') is null --music home log 뽑는 조건
    ),  UNNEST(sectionList) sectionList_item
),
PREPROCESS_FIRST AS (

SELECT transactionId,
       STRUCT(
           JSON_VALUE(sectionInfo, "$.sectionId") AS  sectionId,
           JSON_VALUE(sectionInfo, "$.section") AS  section,
           JSON_VALUE(sectionInfo, "$.index") AS  index,
           JSON_VALUE(sectionInfo, "$.elapsed") AS  elapsed
        ) AS section_info,
        cardIndex,
        cardSize,
        responseStatusCode,
        responseCode,
        domain,
        timestamp
FROM
(
        SELECT  transactionId,
                JSON_EXTRACT(sectionList_item, '$.title') as sectionTitle,
                JSON_EXTRACT(sectionList_item, '$.info') as sectionInfo,
                CAST(JSON_EXTRACT(JSON_EXTRACT(sectionList_item, '$.info'), '$.index') as INT64) as cardIndex,
                cardSize,
                JSON_EXTRACT(sectionList_item, '$.cardList') as sectioncardList,
                If(header is null, 'music_home', header) as domain,
                responseStatusCode,
                responseCode,
                timestamp
        FROM RAW_HOME_LOG
))

SELECT transactionId,
       ARRAY_AGG(section_info IGNORE NULLS) as section_info,
       IF (MIN(cardIndex)> 2, True, FALSE) as cold_start,
       MIN(cardIndex) as smallestIndex,
       CAST(cardSize as INT) as cardSize,
       domain,
       responseStatusCode,
       responseCode,
       timestamp,
       DATE(TIMESTAMP_TRUNC(timestamp, day)) as dt
       
FROM PREPROCESS_FIRST
WHERE transactionId is not Null
GROUP BY transactionId, cardSize, responseStatusCode, responseCode, domain, timestamp
"""

In [2]:
df = bq_to_pandas(query)

In [167]:
df['transactionId'] = df['transactionId'].apply(lambda x: eval(x))

In [ ]:
from skt.gcp import pandas_to_bq, get_bigquery_client

In [ ]:
dest_dataset = "x1112436"
partitioned_dest_table = f"adot_musichome_log_stg"

In [168]:
pandas_to_bq(
             pd_df = df, 
             destination = f"skt-datahub.{dest_dataset}.{partitioned_dest_table}", 
             partition = 'dt',
             overwrite=True
)

In [166]:
# dest_dataset = "x1112436"
# partitioned_dest_table = f"adot_musichome_log_stg"

# get_bigquery_client().query(f"""
#     DROP TABLE IF EXISTS {dest_dataset}.{partitioned_dest_table}"""
# )